## import libraries

In [1]:
! pip install feature-engine

In [34]:
! pip install "xgboost<2.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 41.1 MB/s  0:00:06m0:00:0100:01


In [41]:
import os
import pickle

import numpy as np

import pandas as pd

import boto3

import xgboost as xgb

import sklearn
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

import matplotlib.pyplot as plt

import warnings

In [3]:
## display settings

pd.set_option("display.max_columns", None)

In [4]:
sklearn.set_config(transform_output="pandas")

In [5]:
warnings.filterwarnings("ignore")

## Read the datasets

In [6]:
train = pd.read_csv('train.csv')
val = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')
                  

In [7]:
val

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-03,Kolkata,Banglore,10:10:00,12:55:00,165,0.0,No Info,6565
1,Vistara,2019-03-12,Mumbai,Hyderabad,07:30:00,09:25:00,1555,1.0,No Info,12080
2,Multiple Carriers,2019-05-27,Delhi,Cochin,09:00:00,19:15:00,615,1.0,In-flight meal not included,10197
3,Air India,2019-06-15,Delhi,Cochin,05:15:00,07:40:00,1585,3.0,No Info,10493
4,Jet Airways,2019-03-06,Delhi,Cochin,16:00:00,04:25:00,745,1.0,No Info,15764
...,...,...,...,...,...,...,...,...,...,...
155,Air India,2019-05-06,Kolkata,Banglore,09:50:00,08:55:00,1385,2.0,No Info,13227
156,Spicejet,2019-05-12,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No check-in baggage included,4319
157,Spicejet,2019-05-24,Banglore,Delhi,21:10:00,00:05:00,175,0.0,No check-in baggage included,3919
158,Jet Airways,2019-03-09,Banglore,New Delhi,05:50:00,15:00:00,550,1.0,In-flight meal not included,10037


## preprocessing operations

In [8]:
# airline
air_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
	("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

#doj
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
	("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
	("scaler", MinMaxScaler())
])
#source & destination
location_pipe1 = Pipeline(steps=[
	("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
	("encoder", MeanEncoder()),
	("scaler", PowerTransformer())
])

def is_north(X):
	columns = X.columns.to_list()
	north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
	return (
		X
		.assign(**{
			f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
			for col in columns
		})
		.drop(columns=columns)
	)


location_transformer = FeatureUnion(transformer_list=[
	("part1", location_pipe1),
	("part2", FunctionTransformer(func=is_north))
])


##dep and arrival time 
time_pipe1 = Pipeline(steps=[
	("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
	("scaler", MinMaxScaler())
])

def part_of_day(X, morning=4, noon=12, eve=16, night=20):
	columns = X.columns.to_list()
	X_temp = X.assign(**{
		col: pd.to_datetime(X.loc[:, col]).dt.hour
		for col in columns
	})

	return (
		X_temp
		.assign(**{
			f"{col}_part_of_day": np.select(
				[X_temp.loc[:, col].between(morning, noon, inclusive="left"),
				 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
				 X_temp.loc[:, col].between(eve, night, inclusive="left")],
				["morning", "afternoon", "evening"],
				default="night"
			)
			for col in columns
		})
		.drop(columns=columns)
	)

time_pipe2 = Pipeline(steps=[
	("part", FunctionTransformer(func=part_of_day)),
	("encoder", CountFrequencyEncoder()),
	("scaler", MinMaxScaler())
])


time_transformer = FeatureUnion(transformer_list=[
	("part1", time_pipe1),
	("part2", time_pipe2)
])


## duration
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
	def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
		self.variables = variables
		self.percentiles = percentiles
		self.gamma = gamma


	def fit(self, X, y=None):
		if not self.variables:
			self.variables = X.select_dtypes(include="number").columns.to_list()

		self.reference_values_ = {
			col: (
				X
				.loc[:, col]
				.quantile(self.percentiles)
				.values
				.reshape(-1, 1)
			)
			for col in self.variables
		}

		return self


	def transform(self, X):
		objects = []
		for col in self.variables:
			columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
			obj = pd.DataFrame(
				data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
				columns=columns
			)
			objects.append(obj)
		return pd.concat(objects, axis=1)

def duration_category(X, short=180, med=400):
	return (
		X
		.assign(duration_cat=np.select([X.duration.lt(short),
									    X.duration.between(short, med, inclusive="left")],
									   ["short", "medium"],
									   default="long"))
		.drop(columns="duration")
    )

def is_over(X, value=1000):
	return (
		X
		.assign(**{
			f"duration_over_{value}": X.duration.ge(value).astype(int)
		})
		.drop(columns="duration")
	)

duration_pipe1 = Pipeline(steps=[
	("rbf", RBFPercentileSimilarity()),
	("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
	("cat", FunctionTransformer(func=duration_category)),
	("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
	("part1", duration_pipe1),
	("part2", duration_pipe2),
	("part3", FunctionTransformer(func=is_over)),
	("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
	("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
	("imputer", SimpleImputer(strategy="median")),
	("union", duration_union)
])

# total stops
def is_direct(X):
	return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("", FunctionTransformer(func=is_direct))
])

## additional info
info_pipe1 = Pipeline(steps=[
	("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
	("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
	return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
	("part1", info_pipe1),
	("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
	("union", info_union)
])


## final column transformer

column_transformer = ColumnTransformer(transformers=[
	("air", air_transformer, ["airline"]),
	("doj", doj_transformer, ["date_of_journey"]),
	("location", location_transformer, ["source", 'destination']),
	("time", time_transformer, ["dep_time", "arrival_time"]),
	("dur", duration_transformer, ["duration"]),
	("stops", total_stops_transformer, ["total_stops"]),
	("info", info_transformer, ["additional_info"])
], remainder="passthrough")

## feature selector

estimator = RandomForestRegressor(n_estimators=10, max_depth=3, random_state=42)

selector = SelectBySingleFeaturePerformance(
	estimator=estimator,
	scoring="r2",
	threshold=0.1
) 

## preprocessor
preprocessor = Pipeline(steps=[
	("ct", column_transformer),
	("selector", selector)
])
         


In [9]:
preprocessor.fit(
    train.drop(columns='price'),
    train.price.copy()
)

,steps,"[('ct', ...), ('selector', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('air', ...), ('doj', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [10]:
preprocessor.transform(train.drop(columns="price"))

,air__airline_Jet Airways,doj__date_of_journey_week,doj__date_of_journey_day_of_year,dur__duration_cat,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,0.647059,0.661017,2.0,-0.007496,1.0,0
1,0.0,0.058824,0.067797,2.0,-0.325123,1.0,0
2,1.0,0.823529,0.822034,2.0,0.361360,2.0,0
3,1.0,0.176471,0.169492,2.0,1.416698,2.0,0
4,1.0,0.411765,0.432203,1.0,-0.929637,0.0,1
...,...,...,...,...,...,...,...
635,1.0,0.705882,0.686441,2.0,0.422836,1.0,0
636,1.0,0.588235,0.610169,2.0,0.115456,1.0,0
637,1.0,0.823529,0.796610,2.0,0.351114,1.0,0
638,0.0,0.000000,0.016949,2.0,0.135948,1.0,0


## preprocess the data and upload to bucket
#sagemaker-flt-bucket

In [11]:
BUCKET_NAME = "sagemaker-flt-bucket"

DATA_PREFIX = "data"

In [12]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [13]:
def export_data(data,name,pre):
    # split data into X and y subset
    X= data.drop(columns="price")
    y= data.price.copy()

    ## transforming
    X_pre = pre.transform(X)

    ##exporting
    file_name = get_file_name(name)
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index=False, header = False)
    )

    

In [14]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [15]:
def export_and_upload_bucket(data,name,pre):
    export_data(data,name,pre)
    upload_to_bucket(name)

In [16]:
export_and_upload_bucket(train,"train",preprocessor)

In [17]:
export_and_upload_bucket(val,"val",preprocessor)

In [18]:
export_and_upload_bucket(test,"test",preprocessor)

## Model and Hyperparameter Tuning

In [45]:
session = sagemaker.Session()
region_name = session.boto_region_name

In [46]:
output_path = f"s3://{BUCKET_NAME}/model/output"

In [47]:
model = Estimator(
    image_uri = sagemaker.image_uris.retrieve("xgboost", region_name,"1.2-1"),
    role = sagemaker.get_execution_role(),
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    volume_size = 5,
    output_path= output_path,
    use_spot_instances = True,
    max_run = 300,
    max_wait = 600,
    sagemaker_session = session
)

In [48]:
model.set_hyperparameters(
    objective="reg:squarederror",
    num_round=300,
    eta=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [49]:
hyperparameter_ranges = {
    # Learning rate
    "eta": ContinuousParameter(0.01, 0.3),

    # Tree complexity
    "max_depth": IntegerParameter(3, 10),
    "min_child_weight": ContinuousParameter(1, 20),

    # Regularization
    "alpha": ContinuousParameter(0, 10),     # L1
    "lambda": ContinuousParameter(0, 10),    # L2

    # Row / column sampling
    "subsample": ContinuousParameter(0.5, 1.0),
    "colsample_bytree": ContinuousParameter(0.5, 1.0)
}

In [50]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=25,              # total tuning trials
    max_parallel_jobs=3,      # controls cost + stability
    strategy="Bayesian",
    objective_type="Minimize",
    early_stopping_type="Auto"
)

# Data Channels

In [51]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [52]:
train_data_channel = get_data_channel("train")
train_data_channel

In [53]:
val_data_channel = get_data_channel("val")

In [54]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}

## Train and tune the model

In [55]:
tuner.fit(data_channels)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


....................................................................................................................................................................................................................................................................!


In [56]:
# tuner.best_estmator().deploy()

In [57]:
best_estimator = tuner.best_estimator()


2026-01-18 07:35:15 Starting - Preparing the instances for training
2026-01-18 07:35:15 Downloading - Downloading the training image
2026-01-18 07:35:15 Training - Training image download completed. Training in progress.
2026-01-18 07:35:15 Uploading - Uploading generated training model
2026-01-18 07:35:15 Completed - Training job completed


In [58]:
print(best_estimator.model_data)

s3://sagemaker-flt-bucket/model/output/sagemaker-xgboost-260118-0725-011-68d51daa/output/model.tar.gz


## model Evaluation

In [59]:
with open("xgboost-model", "rb") as f:
    best_model = pickle.load(f)
    
best_model

In [60]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgb.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [61]:
evaluate_model("train")

0.20981407165527344

In [62]:
evaluate_model("val")

0.17531055212020874

In [63]:
evaluate_model("test")

-0.08850181102752686